# doc2vec

In [37]:
import nltk
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [8]:
# Load the dataset
df = pd.read_csv('../01_data/chatbot_extract-1_indexed.csv', sep=',')
df.drop('Ticket Number', axis='columns', inplace=True)
df.drop('count of replies', axis='columns', inplace=True)
df

,ticket,Customer,ticket.1,ticket_state,ticket_template
0,Fehler in der App,Swissmem,"Normenauszug 2018, Rechenbuch Metall, Tabellen...",closed successful,5
1,Fehler in der App,SVBA,Ich kann die Bilder nicht mehr öffnen um mit ...,closed successful,2
2,Fehler in der App,hep,Leider fehlen mir paar Kapitel im Lehrmittel (...,closed successful,5
3,Anmeldung / Aktivierung,AGVS,Ich kann im FK z.b. die Seite 605 nicht lesen ...,closed successful,4
4,Anmeldung / Aktivierung,SVBA,Ich kann das Kapitel Technisches Englisch nich...,closed successful,2
5,Geräteanzahl überschritten,DIHK,Bitte iphone und ipad entfernen.,closed successful,5
6,Fehler in der App,BWZ,"Mein problem ist, dass ich selbst wenn ich mi...",closed_max_devices,4
7,Geräteanzahl überschritten,VELEDES,Nun konnte ich das Buch Spezielle Branchenkund...,closed_waiting_for_reply,5
8,Fehler in der App,hep,Ich kann in der Klasse F(18)-KSS den Bildschir...,closed successful,5
9,Geräteanzahl überschritten,DIHK,Ich würde gerne die Lizenzen bei dem Geräten S...,closed successful,5


Wie viele unterschiedliche Lösungen gibt es?

In [9]:
df['ticket_template'].nunique()

6

In [22]:
# overwrite null values
df = df.fillna('blub')

In [40]:
from nltk.corpus import stopwords
nltk.download('stopwords')
def remove_stop_words(sent):
    stop_words = list(set(stopwords.words('german')))
    return [t.lower() for t in sent if not t in stop_words]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cyrill\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
ticket_texts = df['ticket.1']
templates = df['ticket_template']
tagged_data = []

for i in range(len(ticket_texts)):
    txt = ticket_texts[i].split()
    txt = remove_stop_words(txt)
    tag = templates[i]
    tagged_doc = TaggedDocument(words=txt, tags=[str(tag)])
    tagged_data.append(tagged_doc)

print(len(tagged_data))

81


In [61]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)
for epoch in range(max_epochs):
    #print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

c:\users\cyrill\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  app.launch_new_instance()


In [62]:
tokens = remove_stop_words('problem mit dem i-pad'.split())
new_vec = model.infer_vector(tokens)

model.docvecs.most_similar([new_vec])

[('5', 0.5876184701919556),
 ('1', 0.43549832701683044),
 ('0', 0.3823584020137787),
 ('4', 0.33058086037635803),
 ('3', 0.2891424596309662),
 ('2', 0.05001688003540039)]

In [63]:
tokens = remove_stop_words('problem mit dem iphone'.split())
new_vec = model.infer_vector(tokens)

model.docvecs.most_similar([new_vec])

[('5', 0.5543344020843506),
 ('0', 0.3797021210193634),
 ('4', 0.2797680199146271),
 ('3', 0.2783263325691223),
 ('1', 0.18264834582805634),
 ('2', 0.08005087077617645)]